In [1]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
from torch import nn
import os
import json
import time
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from h3 import h3
from geopy.distance import geodesic as vincenty
from tqdm import tqdm
from collections import OrderedDict


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fine_tuned_lm_trained_save_dir = '/home/uddeshya.singh/level-two_models/Fine-tuned-save'
model_dir =  "/home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir"

tokenizer = AutoTokenizer.from_pretrained(model_dir)

class MultiTaskModel(nn.Module):
    def __init__(self, model_name, num_labels_res8, num_labels_res10, num_labels_res9):
        super(MultiTaskModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size
        self.classifier_res8 = nn.Linear(self.hidden_size, num_labels_res8)
        self.classifier_res10 = nn.Linear(self.hidden_size, num_labels_res10)
        self.classifier_res9 = nn.Linear(self.hidden_size, num_labels_res9)
    
    def forward(self, input_ids, attention_mask=None, labels_res8=None, labels_res10=None, labels_res9=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  
        logits_res8 = self.classifier_res8(pooled_output)
        logits_res10 = self.classifier_res10(pooled_output)
        logits_res9 = self.classifier_res9(pooled_output)

        return {"logits_res10": logits_res10, "logits_res8": logits_res8, "logits_res9": logits_res9}


with open(os.path.join(fine_tuned_lm_trained_save_dir, 'config.json'), 'r') as f:
    config = json.load(f)

num_labels_res8 = config['num_labels_res8']
num_labels_res10 = config['num_labels_res10']
num_labels_res9 = config['num_labels_res9']


model = MultiTaskModel(model_dir, num_labels_res8, num_labels_res10, num_labels_res9)


state_dict_path = "/home/uddeshya.singh/level-two_models/Fine-tuned-save/MM_model.pt"
state_dict = torch.load(state_dict_path, map_location=device)


new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] if k.startswith('module.') else k  
    new_state_dict[name] = v


model.load_state_dict(new_state_dict)


model.to(device)
print("Model loaded successfully!")


Some weights of the model checkpoint at /home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN thi

Model loaded successfully!
